In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
  return false;
}

<IPython.core.display.Javascript object>

In [2]:
# Modules
import scipy as sp
from scipy import sparse, signal, stats
from astropy import convolution as conv
import numpy as np
from numpy.fft import fft, ifft
import matplotlib.pyplot as plt
from matplotlib import colors as clrs
import pandas as pd 
import time
import datetime
import calendar
import sqlite3
from tqdm.autonotebook import tqdm

%matplotlib inline 



# Color definitions
ClrS = (0.74, 0.00, 0.00)
ClrN = (0.20, 0.56, 1.00)

Clr = [(0.00, 0.00, 0.00),
      (0.31, 0.24, 0.00),
      (0.43, 0.16, 0.49),
      (0.32, 0.70, 0.30),
      (0.45, 0.70, 0.90),
      (1.00, 0.82, 0.67)]

# Font Size

font = {'family': 'sans-serif',
        'weight': 'normal',
        'size'   : 21}

plt.rc('font', **font)

C:\Users\Andres\AppData\Local\Temp\ipykernel_428376\1102906597.py:14: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


### Read file with largest interval

In [3]:
db = sqlite3.connect('data\SWAMIS\\100_frames\swamis_00090s_rotAssoc.sdb')
# db = sqlite3.connect('data\SWAMIS\swamisMDI.sdb')
cursor = db.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

# tabs = pd.read_sql_query('SELECT * FROM tabs WHERE frame = 3173',db)
tabs = pd.read_sql_query('SELECT * FROM tabs',db)

[('params',), ('files',), ('tabs',), ('hist',), ('events',), ('event_hist',)]


In [4]:
list(tabs)

['id',
 'frame',
 'scale',
 'assoc_larger',
 'flux',
 'maxB',
 'str_pix',
 'size',
 'x',
 'y',
 'xerr',
 'yerr',
 'closeness',
 'est_dflux',
 'neighbors',
 'xs',
 'ys',
 'lon',
 'lat',
 'size_s']

### Clean elements that have a larger associated element

In [5]:
# Find elements that have an associated larger element
tabs_assLrg = tabs[np.isfinite(tabs['assoc_larger'])].copy().reset_index(drop=True)

# Find maks of all objects with an associated larger element in at least one frame
msk_assLrg = np.isin(tabs['id'],tabs_assLrg['id'])

# Create table with elements without an associated larger element
tabs_NoassLrg = tabs[np.logical_not(msk_assLrg)].copy().reset_index(drop=True)
tabs_NoassLrg

,id,frame,scale,assoc_larger,flux,maxB,str_pix,size,x,y,xerr,yerr,closeness,est_dflux,neighbors,xs,ys,lon,lat,size_s
0,1,1,1,NaN,-409.852418,-77.336472,0,2,2168.542486,3816.000000,0.498192,0.000043,0,-409.852418,None,-66.075702,-891.520053,-14.070494,-73.466620,0.508633
1,2,1,1,NaN,-635.447272,-82.187646,0,3,2046.357913,3811.291415,0.338977,0.321320,1,-635.447272,"-1704,-1701",-4.455403,-889.221020,-0.923948,-73.185057,0.762949
2,3,1,1,NaN,-401.259592,-74.707427,0,2,2042.486745,3810.000000,0.499824,0.000086,1,-401.259592,-1701,-2.502384,-888.572155,-0.515724,-73.077809,0.508633
3,4,1,1,NaN,-744.679039,-95.060790,1,3,1827.657345,3809.082584,0.335591,0.570318,1,-744.679039,-1706,105.836138,-888.242242,21.967109,-72.768028,0.762949
4,5,1,1,NaN,-441.564932,-80.192197,0,2,1823.000000,3810.472332,0.000022,0.499234,1,-441.564932,-1706,108.183968,-888.945966,22.617324,-72.869519,0.508633
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1709783,603934,97,8,NaN,102.070623,33.052047,0,3,313.357983,338.321850,0.338992,0.330350,1,NaN,399340,229.764081,334.812367,14.460829,16.039979,48.948709
1709784,603935,97,8,NaN,64.886713,32.544911,0,2,283.441260,356.000000,0.496538,0.000000,1,NaN,23802,108.920380,406.220358,6.979708,20.563891,32.632473
1709785,603936,97,8,NaN,77.312145,33.833275,0,2,226.510006,373.000000,0.499900,0.000000,1,NaN,23817,-121.044090,474.889072,-8.035034,25.212963,32.632473
1709786,603937,97,8,NaN,77.635614,33.855645,0,2,319.443630,388.000000,0.496812,0.000000,0,NaN,None,254.346051,535.479114,17.816463,29.682017,32.632473


### Calculate lifetimes

In [6]:
# Add life column
tabs_NoassLrg['Life'] = 0

# Calculate lifes
unique_id = np.unique(tabs_NoassLrg['id'])
for i in tqdm(np.arange(0, unique_id.shape[0])):
# for i in np.arange(0,5):
    snglFtr = tabs_NoassLrg[tabs_NoassLrg['id'] == unique_id[i]].copy().reset_index(drop=True)
    tabs_NoassLrg.loc[tabs_NoassLrg['id'] == unique_id[i],'Life'] = (np.max(snglFtr['frame'])-np.min(snglFtr['frame'])+1)



100%|██████████| 488186/488186 [2:10:36<00:00, 62.30it/s]  


### Drop single frame objects and change frame to time

In [7]:
tabs_NoassLrg.to_pickle('data\SWAMIS\\100_frames\swamis_NoAssociate_00090s.pnds')